# ***Trabalho de simulação 1*** 
Yago Alves da Costa - 115212477
<br> Pedro Pinto Moya - 113083474<br> Victor Lisboa - 113102692
<br> Gabriel Silva Pereira - 115192431


## Simuladores e funções auxiliares

In [ ]:
import random
import math
import numpy as np
import matplotlib.pyplot as plt
from math import isclose

# Intervalo de confiança da t-student
def tstudent_ci(mean, variance, samples):
  # Verifica se a media e variancia tem valores nulos (ou bem proximos disso)
  if isclose(mean, 0, abs_tol=1e-9) and isclose(variance, 0, abs_tol=1e-9):
    return 0.0, 0.0, 0.0
  # desvio padrao (raiz quadrada da variancia)
  std_dev = math.sqrt(variance)
  # metade do intervalo
  term = 1.960 * (std_dev/math.sqrt(samples))
  # limite superior do intervalo
  upper_limit = mean + term
  # limite inferior do intervalo
  lower_limit = mean - term
  # precisao do intervalo de confiança (metade do intervalo / centro do intervalo)
  precision = term / mean
  return upper_limit, lower_limit, precision

def simulator_1(maxClientToServe, v_lambda, v_mi):
  round = 0
  simulationTime = 0
  clientsInQueue = 0
  clientsServed = 0
  simulationData = []
  accumlatedClients = 0

  while clientsServed < maxClientToServe:
    # generate random variables in exponential distribution 
    arrivalTime = random.expovariate(v_lambda)
    departureTime = random.expovariate(v_mi)
    if clientsInQueue == 0:
      simulationTime += arrivalTime
      clientsInQueue += 1
    else:
      if arrivalTime < departureTime:
        simulationTime += arrivalTime
        clientsInQueue += 1
      # does not count a service time if no client in queue
      elif clientsInQueue > 0 and departureTime <= arrivalTime:
        simulationTime += departureTime
        clientsServed += 1
        clientsInQueue -= 1
    # get data
    accumlatedClients += clientsInQueue
    simulationData.append((simulationTime, clientsInQueue))
    round += 1
  # given Little' Law L = v_lambda*W
  averageClientsInSystem = accumlatedClients/round
  # W is the average time in the system
  W = averageClientsInSystem/v_lambda
  return W, averageClientsInSystem

def pop(queue):
    elem = queue.pop()
    return elem

def push(queue, x):
    queue.append(x)

def size(queue):
    return len(queue)

def get_exp_rand_num(lambda_val):
    return -math.log(1 - random.random()) / lambda_val

def sum_array(array):
    return sum(array)

def simulator_2(maximo, lambda_val, mi):
    random.seed()
    queue = []
    next_arrival = random.expovariate(lambda_val)
    next_departure = math.inf
    total_wait = 0.0
    customers_serviced = 0
    wait_times = []
    avg_client_system = 0
    acc_client = 0

    while True:
        if next_arrival <= next_departure:
            if size(queue) == 0:
                next_departure = next_arrival + random.expovariate(mi)
            push(queue, next_arrival)
            next_arrival += random.expovariate(lambda_val)
            acc_client += size(queue)
        elif size(queue) > 0:
            wait = next_departure - pop(queue)
            wait_times.append(wait)
            total_wait += wait
            customers_serviced += 1

            if customers_serviced == maximo:
                avg_time_system = sum_array(wait_times)/maximo
                avg_client_system = acc_client/maximo
                return avg_time_system, avg_client_system

            if size(queue) == 0:
                next_departure = math.inf
            else:
                next_departure += random.expovariate(mi)


def scenario(v_lambda, v_mi, maxClients, samples, sim='1'):
  simTimeRounds = []
  simClientsRounds = []
  if sim == '1':
    print('|-------------|')
    print('| Simulator 1 |')
    print('|-------------|')
  elif sim == '2':
    print('|-------------|')
    print('| Simulator 2 |')
    print('|-------------|')
  print('samples')
  print(samples)
  for i in range(samples):
    if sim == '1':
      resSim = simulator_1(maxClients, v_lambda, v_mi)
    elif sim ==  '2':
      resSim = simulator_2(maxClients, v_lambda, v_mi)
    simTimeRounds.append(resSim[0])
    simClientsRounds.append(resSim[1])
  meanTime = np.mean(simTimeRounds)
  varianceTime = np.var(simTimeRounds)
  IC95 = tstudent_ci(meanTime, varianceTime, samples)
  print('mean time')
  print(meanTime)
  print('variance time')
  print(varianceTime)
  print('IC95 for time: upper_limit, lower_limit, precision')
  print(IC95)
  print('---------------------------------------------------')
  meanClient = np.mean(simClientsRounds)
  varianceClient = np.var(simClientsRounds)
  IC95 = tstudent_ci(meanClient, varianceClient, samples)
  print('mean clients')
  print(meanClient)
  print('variance clients')
  print(varianceClient)
  print('IC95 for clients: upper_limit, lower_limit, precision')
  print(IC95)

# ***Cenário 1:***

In [ ]:
scenario(1, 2, 100, 100, '1')
print("#######################################################################")
scenario(1, 2, 100, 100, '2')
print("#######################################################################")
scenario(1, 2, 100, 1000, '1')
print("#######################################################################")
scenario(1, 2, 100, 1000, '2')
print("#######################################################################")
scenario(1, 2, 100, 10000, '1')
print("#######################################################################")
scenario(1, 2, 100, 10000, '2')
print("#######################################################################")

|-------------|
| Simulator 1 |
|-------------|
samples
100
mean time
1.5534950828483938
variance time
0.23397706347216116
IC95 for time: upper_limit, lower_limit, precision
(1.6483025871418122, 1.4586875785549753, 0.061028519072995786)
---------------------------------------------------
mean clients
1.5534950828483938
variance clients
0.23397706347216116
IC95 for clients: upper_limit, lower_limit, precision
(1.6483025871418122, 1.4586875785549753, 0.061028519072995786)
#######################################################################
|-------------|
| Simulator 2 |
|-------------|
samples
100
mean time
0.9549719124363928
variance time
0.1269033906473213
IC95 for time: upper_limit, lower_limit, precision
(1.0247939766790626, 0.8851498481937231, 0.07311425952260171)
---------------------------------------------------
mean clients
1.9998999999999998
variance clients
0.28632298999999994
IC95 for clients: upper_limit, lower_limit, precision
(2.1047779480340836, 1.895022051965916, 0.0

# ***Cenário 2:***

In [ ]:
scenario(2, 4, 100, 100, '1')
print("#######################################################################")
scenario(2, 4, 100, 100, '2')
print("#######################################################################")
scenario(2, 4, 100, 1000, '1')
print("#######################################################################")
scenario(2, 4, 100, 1000, '2')
print("#######################################################################")
scenario(2, 4, 100, 10000, '1')
print("#######################################################################")
scenario(2, 4, 100, 10000, '2')
print("#######################################################################")

|-------------|
| Simulator 1 |
|-------------|
samples
100
mean time
0.7430648024466724
variance time
0.02879662748359733
IC95 for time: upper_limit, lower_limit, precision
(0.7763251578478083, 0.7098044470455365, 0.04476104276722618)
---------------------------------------------------
mean clients
1.4861296048933448
variance clients
0.11518650993438932
IC95 for clients: upper_limit, lower_limit, precision
(1.5526503156956166, 1.419608894091073, 0.04476104276722618)
#######################################################################
|-------------|
| Simulator 2 |
|-------------|
samples
100
mean time
0.4715481538310853
variance time
0.01868853366182545
IC95 for time: upper_limit, lower_limit, precision
(0.4983425321435725, 0.44475377551859807, 0.0568221465714514)
---------------------------------------------------
mean clients
1.9583000000000002
variance clients
0.16147611
IC95 for clients: upper_limit, lower_limit, precision
(2.0370608166651416, 1.879539183334859, 0.040218973939

# ***Cenário 3:***

In [ ]:
scenario(1, 4, 100, 100, '1')
print("#######################################################################")
scenario(1, 4, 100, 100, '2')
print("#######################################################################")
scenario(1, 4, 100, 1000, '1')
print("#######################################################################")
scenario(1, 4, 100, 1000, '2')
print("#######################################################################")
scenario(1, 4, 100, 10000, '1')
print("#######################################################################")
scenario(1, 4, 100, 10000, '2')
print("#######################################################################")

|-------------|
| Simulator 1 |
|-------------|
samples
100
mean time
0.8267509777843455
variance time
0.009501771772670286
IC95 for time: upper_limit, lower_limit, precision
(0.8458564760615144, 0.8076454795071766, 0.02310913296815288)
---------------------------------------------------
mean clients
0.8267509777843455
variance clients
0.009501771772670286
IC95 for clients: upper_limit, lower_limit, precision
(0.8458564760615144, 0.8076454795071766, 0.02310913296815288)
#######################################################################
|-------------|
| Simulator 2 |
|-------------|
samples
100
mean time
0.32864950766938883
variance time
0.0030065574535995464
IC95 for time: upper_limit, lower_limit, precision
(0.33939659616523177, 0.3179024191735459, 0.032700759456649485)
---------------------------------------------------
mean clients
1.3202
variance clients
0.012525960000000004
IC95 for clients: upper_limit, lower_limit, precision
(1.3421362093206644, 1.2982637906793357, 0.01661

# ***Cenário 4:***

Ao adaptar o simulador 1 para um contexto determinístico ele irá parar de funcionar por conta da forma como se decide se o evento é de partida ou de chegada. Dado que se colhe de forma aleatória as amostras tanto de evento de partida quanto de evento de chegada. Nesse sentido, o menor tempo é escolhido para ser o evento a ser avaliado na rodada. Num contexto aleatório, hora a partida e hora a chegada é um evento avaliado. Num cenário determinístico, ou só as chegadas serão avaliadas ou só as partidas serão avaliadas. Ou seja, ou apenas gero eventos de partida ou só gero eventos de chegada fixando um valor para o tempo de chegada ou de partida, ao invés de atrelá-los a um número aleatório.

O tempo médio de espera dos clientes ...

A quantidade média de clientes no sistema ...